# Context Reranking & Custom Meta Functions Demo

This notebook demonstrates:
1. LLM-based context reranking by relevance
2. Custom meta functions for filtering, ranking, and transforming context

In [1]:
# Enable chat mode
%llm_chat on

💬 Chat mode: ON
📓 Notebook context mode: ON
📝 Just type in any cell to chat!
💡 Your notebook cells are now the LLM's context window!


## Part 1: Build Some Context

Let's create a variety of cells with different topics:

In [2]:
# Cell about Python basics
def hello_world():
    print("Hello, World!")
    
hello_world()

In [2]:
# Cell about machine learning
import numpy as np

# Simple linear regression example
X = np.array([1, 2, 3, 4, 5])
y = 2 * X + 1
print(f"Linear relationship: y = 2x + 1")
print(f"X: {X}")
print(f"y: {y}")

In [2]:
What is gradient descent and how does it work?

In [ ]:
# Cell about web development
html_template = """
<html>
  <head><title>My Page</title></head>
  <body>
    <h1>Welcome!</h1>
    <p>This is a simple HTML page.</p>
  </body>
</html>
"""
print("Basic HTML structure created")

In [ ]:
# Random cell about cooking
recipe = {
    "name": "Chocolate Chip Cookies",
    "ingredients": ["flour", "butter", "sugar", "eggs", "chocolate chips"],
    "time": "30 minutes"
}
print(f"Recipe: {recipe['name']}")

In [2]:
Can you explain neural networks?

## Part 2: LLM-Based Reranking

Now let's rerank cells by relevance to a specific query:

In [2]:
# First, let's see the current context order
%llm_context

In [2]:
# Rerank by relevance to machine learning
%llm_rerank "machine learning and neural networks"

In [2]:
# Check the new order
%llm_context

In [ ]:
# Just show ranking without reordering
%llm_rerank --show "web development"

In [ ]:
# Keep only top 5 most relevant cells
%llm_rerank --top=5 "Python programming"

In [ ]:
# Clear reranking to restore original order
%llm_rerank_clear

## Part 3: Custom Meta Functions

Define your own context processing logic:

In [ ]:
%%meta filter
def filter_cells(messages):
    """Filter out cells that are too short or contain specific keywords."""
    filtered = []
    
    for msg in messages:
        content = msg['content']
        
        # Skip very short messages
        if len(content) < 20:
            continue
            
        # Skip cells about cooking (as an example)
        if 'recipe' in content.lower() or 'cooking' in content.lower():
            continue
            
        filtered.append(msg)
    
    return filtered

In [ ]:
%%meta ranking
def rank_cells(messages, query):
    """Custom ranking based on keyword matching."""
    # Simple keyword-based ranking
    query_words = set(query.lower().split())
    
    scored_messages = []
    for msg in messages:
        content_words = set(msg['content'].lower().split())
        score = len(query_words.intersection(content_words))
        scored_messages.append((score, msg))
    
    # Sort by score (descending)
    scored_messages.sort(key=lambda x: x[0], reverse=True)
    
    # Return reordered messages
    return [msg for score, msg in scored_messages]

In [ ]:
%%meta transform
def transform_context(messages):
    """Add metadata to each message."""
    transformed = []
    
    for i, msg in enumerate(messages):
        # Add position and length metadata
        new_msg = msg.copy()
        new_msg['content'] = f"[Position {i+1}, Length {len(msg['content'])}]\n{msg['content']}"
        transformed.append(new_msg)
    
    return transformed

In [ ]:
# List defined meta functions
%llm_meta_list

In [ ]:
# Apply the filter function
%llm_apply_meta filter

In [ ]:
# Apply custom ranking
%llm_apply_meta ranking "machine learning neural networks"

In [ ]:
# Apply all meta functions in sequence
%llm_apply_meta all "Python programming"

In [ ]:
# Check the transformed context
%llm_context

## Part 4: Advanced Meta Functions

More sophisticated examples:

In [ ]:
%%meta ranking
import re
from collections import Counter

def rank_cells(messages, query):
    """Semantic ranking using TF-IDF-like scoring."""
    # Tokenize query
    query_tokens = re.findall(r'\w+', query.lower())
    query_freq = Counter(query_tokens)
    
    scored = []
    for msg in messages:
        # Tokenize message
        msg_tokens = re.findall(r'\w+', msg['content'].lower())
        msg_freq = Counter(msg_tokens)
        
        # Calculate similarity score
        score = 0
        for token, freq in query_freq.items():
            if token in msg_freq:
                # Higher score for more occurrences
                score += min(freq, msg_freq[token]) * (1 + len(token) / 10)
        
        # Boost score for assistant responses about the topic
        if msg['role'] == 'assistant' and any(t in msg_tokens for t in query_tokens):
            score *= 1.5
            
        scored.append((score, msg))
    
    # Sort by score
    scored.sort(key=lambda x: x[0], reverse=True)
    
    # Return reordered, keeping at least some context
    result = [msg for _, msg in scored]
    
    # Ensure we have at least 3 messages
    if len(result) < 3 and len(messages) >= 3:
        return result + messages[:3-len(result)]
    
    return result

In [ ]:
# Apply the new ranking
%llm_apply_meta ranking "neural networks gradient descent"

## Summary

You now have powerful tools for context management:

1. **LLM-based reranking** - Let the AI intelligently reorder cells by relevance
2. **Custom meta functions** - Define your own logic for filtering, ranking, and transforming
3. **Flexible application** - Apply functions individually or in sequence

This gives you complete control over what context the LLM sees and in what order!

In [ ]:
# Disable chat mode
%llm_chat off